# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kerikeri,-35.2268,173.9474,9.62,80,51,2.84,NZ,1690389105
1,1,labytnangi,66.6572,66.4183,16.88,95,100,3.97,RU,1690388769
2,2,bandarbeyla,9.4942,50.8122,24.11,82,9,14.73,SO,1690388739
3,3,qaqortoq,60.7167,-46.0333,6.82,84,100,2.75,GL,1690388593
4,4,mckenzie,47.7084,-103.4477,27.79,30,0,0.00,US,1690388910


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_humidity =city_data_df.hvplot.points("Lng", "Lat", geo= True, tiles=True , size ="Humidity")

# Display the map
display(map_humidity)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Wind speed > 5, humidity > 90 , temp > 100
fantastic_weather = city_data_df[city_data_df['Humidity'] > 90]
fantastic_weather = fantastic_weather[fantastic_weather['Max Temp'] > 27]
fantastic_weather = fantastic_weather[fantastic_weather['Wind Speed'] > 1]
# Drop any rows with null values
fantastic_weather = fantastic_weather.dropna()


# Display sample data
display(fantastic_weather)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
92,92,keelung,25.1283,121.7419,29.49,92,100,13.68,TW,1690389104
161,161,inarajan village,13.2736,144.7484,27.54,96,100,9.45,GU,1690388738
339,339,saipan,15.1355,145.7010,27.39,100,40,7.20,MP,1690389211
434,434,hengchun,22.0042,120.7439,27.68,91,100,18.89,TW,1690389239
503,503,bodden town,19.2833,-81.2500,27.21,96,75,2.57,KY,1690389262


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = fantastic_weather.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
display(hotel_df)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
92,92,keelung,25.1283,121.7419,29.49,92,100,13.68,TW,1690389104,
161,161,inarajan village,13.2736,144.7484,27.54,96,100,9.45,GU,1690388738,
339,339,saipan,15.1355,145.7010,27.39,100,40,7.20,MP,1690389211,
434,434,hengchun,22.0042,120.7439,27.68,91,100,18.89,TW,1690389239,
503,503,bodden town,19.2833,-81.2500,27.21,96,75,2.57,KY,1690389262,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
    
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
keelung - nearest hotel: 華國商務飯店
inarajan village - nearest hotel: No hotel found
saipan - nearest hotel: Chalan Kanoa Beach Hotel
hengchun - nearest hotel: 海的顏色精品旅館
bodden town - nearest hotel: Turtle Nest Inn


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
92,92,keelung,25.1283,121.7419,29.49,92,100,13.68,TW,1690389104,華國商務飯店
161,161,inarajan village,13.2736,144.7484,27.54,96,100,9.45,GU,1690388738,No hotel found
339,339,saipan,15.1355,145.7010,27.39,100,40,7.20,MP,1690389211,Chalan Kanoa Beach Hotel
434,434,hengchun,22.0042,120.7439,27.68,91,100,18.89,TW,1690389239,海的顏色精品旅館
503,503,bodden town,19.2833,-81.2500,27.21,96,75,2.57,KY,1690389262,Turtle Nest Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_hotel =hotel_df.hvplot.points("Lng", "Lat", geo= True, tiles=True , scale = 1, hover_cols =['Hotel Name', 'Country'] )

# Display the map
display(map_hotel)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)